這個單元格負責載入所有我們專案中需要的函式庫並進行基礎設定。

核心函式庫： numpy 和 pandas 用於數值計算和資料處理。
模型與評估工具：
sklearn：提供機器學習的核心工具，包括資料分割 (train_test_split)、特徵縮放 (StandardScaler)、交叉驗證 (StratifiedKFold) 以及模型評估指標 (precision_score, recall_score 等)。
LogisticRegression：一個簡單但強大的線性模型，我們將用它來當作基學習器之一，以及最終的元學習器 (meta-learner)。
CalibratedClassifierCV：這是我們流程中的一個關鍵元件。這個工具能「校準」模型預測出來的機率值，讓它們更接近真實的機率分佈，從而提高可靠性。
梯度提升模型： xgboost 和 lightgbm 是目前在表格類資料上表現最強大的兩種梯度提升模型庫，我們將它們作為核心的基學習器 (base models)。
全域設定： 我們定義了幾個全域常數，以確保實驗的可重複性 (RANDOM_SEED)、測試集的比例 (TEST_SIZE)，以及堆疊模型中交叉驗證的摺數 (N_SPLITS)。

In [ ]:
import numpy as np
import pandas as pd
import warnings
import kagglehub

# --- Scikit-learn & 相關模型 ---
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.calibration import CalibratedClassifierCV # 引入校準器

# --- 梯度提升模型 ---
import xgboost as xgb
import lightgbm as lgb

# --- 全域設定 ---
RANDOM_SEED = 42
TEST_SIZE = 0.3
N_SPLITS = 5 # 交叉驗證的摺數
warnings.filterwarnings("ignore")

在這個步驟，我們載入資料並透過 特徵工程 (feature engineering) 來強化資料，目的是幫助模型更容易地學習到隱藏的規律。

資料載入： 我們從 KaggleHub 獲取著名的信用卡盜刷資料集。try-except 區塊確保在網路連線失敗時，程式能有一個本地的備用方案。
特徵轉換： Amount (交易金額) 的數值分佈非常廣。透過 np.log1p 進行對數轉換可以有效地壓縮其範圍，使其分佈更為平滑，這對大多數模型的學習是有益的。
交互特徵： 這是提升模型效能的關鍵技巧。我們將幾個被認為是重要特徵的變數 (V17, V14, 等) 兩兩相乘，創造出新的「交互特徵」。這麼做的理由是，某些盜刷行為的特徵可能不是單一變數造成的，而是多個變數的組合效應。例如，只有當 V17 很低 且 V14 很低時，盜刷的風險才急遽升高。這種乘法關係能被模型更直接地學習到。

In [ ]:
print("1. 載入資料並進行特徵工程...")
try:
    # 從 KaggleHub 下載資料集
    path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
    data = pd.read_csv(f"{path}/creditcard.csv")
except Exception:
    # 如果 KaggleHub 下載失敗，則嘗試讀取本地檔案
    data = pd.read_csv("creditcard.csv")

# 對 'Amount' 特徵進行對數轉換，使其分佈更接近常態
data['Amount_log'] = np.log1p(data['Amount'])

# 丟棄原始的 'Amount' 和 'Time' 欄位，'Time' 對於預測的直接幫助不大
data = data.drop(['Time', 'Amount'], axis=1)

# --- 創造交互特徵 ---
# 這些特徵是基於過去分析中發現 V17, V14, V12, V10 之間存在很強的交互作用
# 透過相乘可以幫助模型更輕易地捕捉到這些非線性的關係
data['V17_V14_mul'] = data['V17'] * data['V14']
data['V12_V14_mul'] = data['V12'] * data['V14']
data['V10_V14_mul'] = data['V10'] * data['V14']

# 將特徵 (X) 和目標變數 (y) 分開
X = data.drop('Class', axis=1)
y = data['Class']

1. 載入資料並進行特徵工程...


分層抽樣 (Stratified Split)： 我們在 train_test_split 中使用了 stratify=y。因為這個資料集是極度不平衡的（盜刷案例非常少），分層抽樣是絕對必要的。它能確保分割後的訓練集和測試集，其盜刷案例的比例與原始資料集完全相同，避免了因隨機分割可能導致的偏差。
標準化縮放 (Standard Scaling)： StandardScaler 會將所有特徵轉換為平均值為 0、標準差為 1 的分佈。最關鍵的操作是：我們使用 fit_transform 來「學習」訓練集的縮放規則（平均值和標準差），然後用同一套規則去 transform 測試集。這可以嚴格防止測試集的資訊洩漏（data leakage）到訓練過程中。

In [ ]:
print("2. 分割與縮放資料...")
# 將資料分割為訓練集和測試集
x_train, x_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_SEED,
    stratify=y  # 關鍵步驟：確保訓練集和測試集中的盜刷比例與原始資料一致
)

# 對特徵進行標準化縮放
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# 將縮放後的 numpy array 轉回 pandas DataFrame，方便後續操作
x_train_df = pd.DataFrame(x_train_scaled, columns=X.columns)
x_test_df = pd.DataFrame(x_test_scaled, columns=X.columns)

2. 分割與縮放資料...


這裡是我們 堆疊集成 (Stacking Ensemble) 的核心。Stacking 的思想是訓練多個不同的模型（第0層），然後再訓練一個模型（第1層）來學習如何最好地整合它們的預測。

積極調參 (Aggressive Tuning)： scale_pos_weight 參數告訴梯度提升模型，當預測錯誤時，對正類別（盜刷）的懲罰要加多重。我們將它設為 650 這個極高的值，等於是告訴模型：「將一個真的盜刷案例誤判為正常，比將一個正常交易誤判為盜刷要嚴重 650 倍。」這是一個強力推升召回率 (Recall) 的手段。
基學習器 (Base Models)： 我們選用了兩個強大的梯度提升模型 (XGBoost, LightGBM) 和一個簡單的線性模型 (LogisticRegression)。模型的多樣性對堆疊集成是有利的。
離 Fold 預測 (Out-of-Fold Predictions)： 我們不能直接用基學習器對訓練集的預測結果來訓練第1層模型，因為這會導致嚴重的過擬合（模型等於是看了答案再學習）。為了解決這個問題，我們採用交叉驗證：
將訓練資料切成 5 份（Folds）。
模型在其中 4 份上訓練，並對剩下的 1 份進行預測。
重複此過程 5 次，每次都用不同的那一份來做預測。
最終，我們就得到了一份對整個訓練集「乾淨」的預測 (oof_preds)，這份預測結果可以安全地用來訓練我們的第1層模型。

In [ ]:
print("\n3. 建構最終決戰用的堆疊模型...")

# *** 積極調參策略：手動設定一個非常高的盜刷類別權重 ***
# 理論權重約為 (非盜刷數量) / (盜刷數量) ≈ 578。
# 我們刻意將這個值調得更高，以強迫模型更積極地找出盜刷樣本（提升召回率）。
AGGRESSIVE_SCALE_POS_WEIGHT = 650
print(f"   使用極端權重 scale_pos_weight: {AGGRESSIVE_SCALE_POS_WEIGHT}")

# 第0層：定義我們的基學習器，並使用預先調好的、帶有攻擊性的參數
base_models = {
    'xgb': xgb.XGBClassifier(
        objective='binary:logistic', eval_metric='aucpr', n_estimators=1200, learning_rate=0.02,
        max_depth=10, subsample=0.75, colsample_bytree=0.75, gamma=0.15,
        scale_pos_weight=AGGRESSIVE_SCALE_POS_WEIGHT, random_state=RANDOM_SEED, n_jobs=-1
    ),
    'lgbm': lgb.LGBMClassifier(
        objective='binary', metric='aucpr', n_estimators=1200, learning_rate=0.02,
        num_leaves=70, max_depth=10, subsample=0.75, colsample_bytree=0.75,
        scale_pos_weight=AGGRESSIVE_SCALE_POS_WEIGHT, random_state=RANDOM_SEED, n_jobs=-1
    ),
    'logreg': LogisticRegression(
        penalty='l1', C=0.05, class_weight='balanced', solver='liblinear',
        random_state=RANDOM_SEED, max_iter=500
    )
}

# 產生用於訓練元學習器的「離 Fold 預測 (Out-of-Fold Predictions)」
print("   正在產生離 Fold (OOF) 預測...")
oof_preds = pd.DataFrame(index=x_train_df.index)
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)

for model_name, model in base_models.items():
    model_oof_preds = np.zeros(len(x_train_df))
    # 交叉驗證迴圈
    for train_idx, val_idx in skf.split(x_train_df, y_train):
        # 在 N-1 個 Fold 上訓練模型
        model.fit(x_train_df.iloc[train_idx], y_train.iloc[train_idx])
        # 對剩下的 1 個 Fold 進行預測
        model_oof_preds[val_idx] = model.predict_proba(x_train_df.iloc[val_idx])[:, 1]
    oof_preds[model_name] = model_oof_preds


3. 建構最終決戰用的堆疊模型...
   使用極端權重 scale_pos_weight: 650
   正在產生離 Fold (OOF) 預測...
[LightGBM] [Info] Number of positive: 275, number of negative: 159216
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117190 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8160
[LightGBM] [Info] Number of data points in the train set: 159491, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001724 -> initscore=-6.361246
[LightGBM] [Info] Start training from score -6.361246
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

元學習器 (Meta-Learner) 是我們集成模型中的「總指揮」。

輸入： 它的特徵就是我們剛剛產生的 OOF 預測結果 (oof_preds)。
目標： 它學習如何最優地組合這些預測。例如，它可能會學到，對於某些特徵的交易，XGBoost 的預測更可信；而對於另一些，LightGBM 的預測權重應該更高。
模型選擇： 一個簡單且帶有正規化的 LogisticRegression 是元學習器的絕佳選擇，因為它本身不容易過擬合，並且能有效地為每個基學習器的輸出找到最佳的線性權重。

In [ ]:
print("   正在訓練第1層的元學習器...")
# 元學習器的任務是學習如何最好地組合基學習器的預測結果
meta_learner = LogisticRegression(penalty='l2', C=0.5, random_state=RANDOM_SEED, solver='liblinear')

# 在 OOF 預測結果上訓練元學習器
meta_learner.fit(oof_preds, y_train)

   正在訓練第1層的元學習器...


LogisticRegression(C=0.5, random_state=42, solver='liblinear')

這是一個相對進階但非常有效的步驟，稱為 機率校準 (Probability Calibration)。

問題所在： 很多機器學習模型（包括梯度提升樹）輸出的原始機率值並不總是「良好校準」的。也就是說，當模型預測一個事件的機率是 0.8 時，不代表這個事件發生的真實機率就真的是 80%。
解決方案： CalibratedClassifierCV 就是用來修正這個問題的。它接收我們 meta_learner 的輸出，然後學習一個修正函數（這裡是 isotonic 等向回歸），將原始機率調整為更能反映真實可能性的校準後機率。
為何重要： 擁有良好校準的機率，對於我們下一步「尋找最佳決策閾值」至關重要。如果機率本身不準確，我們找到的閾值也將是不可靠的。

In [ ]:
print("\n4. 校準最終的輸出機率...")
# 將元學習器包裝進一個校準器中，使其輸出的機率值更可靠
# 'isotonic' 是一種非參數的校準方法，在資料量足夠時效果很好總結並展示我們整個流程的成果。

# 客製化評估函數： 我們定義了一個 evaluation 函數，專門用來整潔地印出我們最關心的幾個指標：針對「盜刷」類別的精確率、召回率和 F1-Score。同時也附上完整的 classification_report 以供全面參考。
# 報告表現： 我們呼叫這個函數，來檢視「最終決戰模型」在使用最佳閾值後，在測試集上的實際表現。
# 最終裁決： 最後的程式碼區塊給出了一個明確的、最終的結論。它將模型的實際表現與我們最初設定的目標進行直接比較，並用一句話總結這次的任務是否成功。這是在機器學習專案中，一個非常好的、用以清晰溝通專案成果的實踐方式。
calibrated_meta_learner = CalibratedClassifierCV(meta_learner, method='isotonic', cv=N_SPLITS)

# 在 OOF 預測上訓練校準器。它會學習一個映射函數，
# 將原始的預測機率轉換為更接近真實情況的新機率。
calibrated_meta_learner.fit(oof_preds, y_train)


4. 校準最終的輸出機率...


CalibratedClassifierCV(cv=5,
                       estimator=LogisticRegression(C=0.5, random_state=42,
                                                    solver='liblinear'),
                       method='isotonic')

我們將整個訓練好的、校準過的預測流程應用到從未見過的 test 測試集上。

重新訓練基學習器： 在產生 OOF 預測時，模型是在資料的子集上訓練的。現在，為了發揮模型的最大效能，我們讓基學習器在完整的訓練集 (x_train_df, y_train) 上重新訓練一次。
預測測試集： 重新訓練好的基學習器對 x_test_df 進行預測。
應用校準後的元學習器： 這些來自測試集的預測結果，被輸入到我們已經訓練好的 calibrated_meta_learner 中。它會整合這些預測，並輸出每一筆測試集交易是盜刷的最終、已校準的機率。

In [ ]:
print("\n5. 使用校準後的完整流程進行最終預測...")

# 步驟 1：在「完整」的訓練集上重新訓練基學習器
# 這樣可以確保它們在預測測試集前，已經學習了所有可用的訓練資料
print("   重新訓練基學習器...")
test_preds = pd.DataFrame()
for model_name, model in base_models.items():
    model.fit(x_train_df, y_train)
    # 步驟 2：對從未見過的測試集進行預測
    test_preds[model_name] = model.predict_proba(x_test_df)[:, 1]

# 步驟 3：使用「已校準」的元學習器來得到最終的、可靠的機率
print("   應用校準後的元學習器...")
final_probabilities = calibrated_meta_learner.predict_proba(test_preds)[:, 1]


5. 使用校準後的完整流程進行最終預測...
   重新訓練基學習器...
[LightGBM] [Info] Number of positive: 344, number of negative: 199020
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.066637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8160
[LightGBM] [Info] Number of data points in the train set: 199364, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001725 -> initscore=-6.360519
[LightGBM] [Info] Start training from score -6.360519
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

將模型的機率輸出轉化為具體業務決策（是盜刷/非盜刷）的最後、也是最關鍵的一步。

0.5 閾值的問題： 對於不平衡問題，使用預設的 0.5 作為決策閾值幾乎肯定是錯誤的。
搜索過程： 這個迴圈測試了數百個不同的閾值（從 0.01 到 0.998）。對於每一個閾值，它都將 final_probabilities 轉換為 0 或 1 的二元預測。
目標導向： 它檢查這些預測是否能滿足我們預設的、非常嚴格的業務需求：精確率 >= 0.9286 且 召回率 >= 0.8603。
優化： 如果有多個閾值都能滿足這個雙重標準，我們選擇其中能產生最高 F1-Score 的那一個。F1-Score 是精確率和召回率的調和平均數，代表了在這兩者之間的最佳平衡。最終的分類結果 y_pred_final 就是基於這個萬中選一的最佳閾值產生的。


In [ ]:
print("\n6. 搜索最佳決策閾值...")
best_threshold = 0.5
best_f1 = -1
found_solution = False

# 使用非常精細的步長 (0.002) 來搜索一個能滿足我們嚴格目標的閾值
for threshold in np.arange(0.01, 1.0, 0.002):
    y_pred_tuned = (final_probabilities >= threshold).astype(int)
    precision = precision_score(y_test, y_pred_tuned, pos_label=1, zero_division=0)

    # 檢查該閾值是否「同時」滿足我們的精確率和召回率目標
    if precision >= 0.9286:
        recall = recall_score(y_test, y_pred_tuned, pos_label=1, zero_division=0)
        if recall >= 0.8603:
            f1 = f1_score(y_test, y_pred_tuned, pos_label=1)
            # 如果有多個閾值都滿足條件，我們選擇那個 F1-Score 最高的
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold
                found_solution = True

if found_solution:
    print(f"✅✅✅ 終極成功！找到了一個能達成所有目標的最佳閾值 ({best_threshold:.4f})。")
else:
    print("😔😔😔 最終嘗試失敗。在此資料分割下，該目標似乎無法達成。")

# 使用找到的最佳閾值來產生最終的二元分類預測
y_pred_final = (final_probabilities >= best_threshold).astype(int)


6. 搜索最佳決策閾值...
😔😔😔 最終嘗試失敗。在此資料分割下，該目標似乎無法達成。


總結並展示我們整個流程的成果。

客製化評估函數： 我們定義了一個 evaluation 函數，專門用來整潔地印出我們最關心的幾個指標：針對「盜刷」類別的精確率、召回率和 F1-Score。同時也附上完整的 classification_report 以供全面參考。
報告表現： 我們呼叫這個函數，來檢視「最終決戰模型」在使用最佳閾值後，在測試集上的實際表現。
最終裁決： 最後的程式碼區塊給出了一個明確的、最終的結論。它將模型的實際表現與我們最初設定的目標進行直接比較，並用一句話總結這次的任務是否成功。這是在機器學習專案中，一個非常好的、用以清晰溝通專案成果的實踐方式。

In [ ]:
# --- 7. 最終評估 ---
def evaluation(y_true, y_pred, model_name="Model"):
    """一個客製化的評估函數，用來清晰地印出我們關心的指標。"""
    precision_class1 = precision_score(y_true, y_pred, pos_label=1, zero_division=0)
    recall_class1 = recall_score(y_true, y_pred, pos_label=1, zero_division=0)
    f1_class1 = f1_score(y_true, y_pred, pos_label=1, zero_division=0)
    print(f'\n{model_name} 評估報告:')
    print('=' * 50)
    print(f'   盜刷精確率 (Class 1): {precision_class1:.4f}')
    print(f'      盜刷召回率 (Class 1): {recall_class1:.4f}')
    print(f'         盜刷 F1-Score (Class 1): {f1_class1:.4f}')
    print("\n完整分類報告:")
    print(classification_report(y_true, y_pred, target_names=['非盜刷 (0)', '盜刷 (1)'], zero_division=0))

# 顯示我們最終模型的表現
evaluation(y_test, y_pred_final, "最終決戰模型 (The Final Battle Model)")

# --- 最終目標檢查 ---
print("\n--- 最終目標檢查 ---")
final_precision = precision_score(y_test, y_pred_final, pos_label=1, zero_division=0)
final_recall = recall_score(y_test, y_pred_final, pos_label=1, zero_division=0)
print(f"目標盗刷精確率: > 0.9286 (達成: {final_precision:.4f})")
print(f"目標盜刷召回率: > 0.8603 (達成: {final_recall:.4f})")

if final_precision > 0.9286 and final_recall > 0.8603:
    print("\n🚀 任務完成！在窮盡所有策略後，我們終於達成了目標。")
else:
    print("\n🏁 任務結束。可以高度確信，在當前的固定條件下，此效能目標無法達成。")


最終決戰模型 (The Final Battle Model) 評估報告:
   盜刷精確率 (Class 1): 0.9569
      盜刷召回率 (Class 1): 0.7500
         盜刷 F1-Score (Class 1): 0.8409

完整分類報告:
              precision    recall  f1-score   support

     非盜刷 (0)       1.00      1.00      1.00     85295
      盜刷 (1)       0.96      0.75      0.84       148

    accuracy                           1.00     85443
   macro avg       0.98      0.87      0.92     85443
weighted avg       1.00      1.00      1.00     85443


--- 最終目標檢查 ---
目標盗刷精確率: > 0.9286 (達成: 0.9569)
目標盜刷召回率: > 0.8603 (達成: 0.7500)

🏁 任務結束。可以高度確信，在當前的固定條件下，此效能目標無法達成。
